In [ ]:
# Stage-I ckpt path 
stage_1_path = "./checkpoints/stage1.pt"
stage_2_path = "./checkpoints/stage2.pt"
save_dir="vis_270p_1080p"
# 2 ~ 3
shift_t = 2.5
# 4 ~ 6
sample_step = 5
# 10 ~ 13
cfg_second = 13
 # 650 ~ 750
deg_latent_strength=675
# stage_1_hw 

#TODO Stage I CFG here
cfg_first = 8


In [ ]:
import os
current_directory = os.getcwd()
os.chdir(os.path.dirname(current_directory))
new_directory = os.getcwd()
print(f"working directory: {new_directory}")

In [ ]:

import os
import argparse
import torch
import numpy as np
import copy

from sat.model.base_model import get_model
from arguments import get_args
from torchvision.io.video import write_video

from flow_video import FlowEngine
from diffusion_video import SATVideoDiffusionEngine

import os
from utils import disable_all_init, decode, prepare_input, save_memory_encode_first_stage, save_mem_decode, seed_everything
disable_all_init()


def init_model(model, second_model, args, second_args):
    share_cache = dict()
    second_share_cache = dict()
    if hasattr(args, 'share_cache_config'):
        for k, v in args.share_cache_config.items():
            share_cache[k] = v
    if hasattr(second_args, 'share_cache_config'):
        for k, v in second_args.share_cache_config.items():
            second_share_cache[k] = v

    for n, m in model.named_modules():
        m.share_cache = share_cache        
        if hasattr(m, "register_new_modules"):
            m.register_new_modules()
    for n, m in second_model.named_modules():
        m.share_cache = second_share_cache        
        if hasattr(m, "register_new_modules"):
            m.register_new_modules()        

    weight_path = args.inf_ckpt
    weight = torch.load(weight_path, map_location="cpu")
    if "model.diffusion_model.mixins.pos_embed.freqs_sin" in weight["module"]:
        del weight["module"]["model.diffusion_model.mixins.pos_embed.freqs_sin"]
        del weight["module"]["model.diffusion_model.mixins.pos_embed.freqs_cos"]
    msg = model.load_state_dict(weight["module"], strict=False)
    print(msg)
    second_weight_path = args.inf_ckpt2
    second_weight = torch.load(second_weight_path, map_location="cpu")
    if "model.diffusion_model.mixins.pos_embed.freqs_sin" in second_weight["module"]:
        del second_weight["module"]["model.diffusion_model.mixins.pos_embed.freqs_sin"]
        del second_weight["module"]["model.diffusion_model.mixins.pos_embed.freqs_cos"]
    second_msg = second_model.load_state_dict(second_weight["module"], strict=False)
    print(second_msg)

def get_first_results(model, text, num_frames, H, W, neg_prompt=None):
    """Get first Stage results.

    Args:
        model (nn.Module): first stage model.
        text (str): text prompt
        num_frames (int): number of frames
        H (int): height of the first stage results
        W (int): width of the first stage results
        neg_prompt (str): negative prompt

    Returns:
        Tensor: first stage video.
    """
    device = 'cuda'
    T = 1 + (num_frames - 1) // 4
    F = 8
    motion_text_prefix = [
        'very low motion,',
        'low motion,',
        'medium motion,',
        'high motion,',
        'very high motion,',
    ]
    pos_prompt = ""
    if neg_prompt is None:
        neg_prompt = ""
    with torch.no_grad():
        model.to('cuda')
        input_negative_prompt = motion_text_prefix[
            0] + ', ' + motion_text_prefix[1] + neg_prompt
        c, uc = prepare_input(text,
                              model,
                              T,
                              negative_prompt=input_negative_prompt,
                              pos_prompt=pos_prompt)
        with torch.no_grad(), torch.amp.autocast(enabled=True,
                                                 device_type='cuda',
                                                 dtype=torch.bfloat16):
            samples_z = model.sample(
                c,
                uc=uc,
                batch_size=1,
                shape=(T, 16, H // F, W // F),
                num_steps=model.share_cache.get('first_sample_step', None),
            )
        samples_z = samples_z.permute(0, 2, 1, 3, 4).contiguous()

        model.to('cpu')
        torch.cuda.empty_cache()
        first_stage_model = model.first_stage_model
        first_stage_model = first_stage_model.to(device)

        latent = 1.0 / model.scale_factor * samples_z

        samples = decode(first_stage_model, latent)
    model.to('cpu')
    return samples
def get_second_results(model, text, first_stage_samples, num_frames):
    """Get second Stage results.

    Args:
        model (nn.Module): second stage model.
        text (str): text prompt
        first_stage_samples (Tensor): first stage results
        num_frames (int): number of frames
    Returns:
        Tensor: second stage results.
    """

    t, h, w, c = first_stage_samples.shape
    first_stage_samples = first_stage_samples[:num_frames]
    first_stage_samples = (first_stage_samples / 255.)
    first_stage_samples = (first_stage_samples - 0.5) / 0.5

    target_size = model.share_cache.get('target_size', None)
    if target_size is None:
        upscale_factor = model.share_cache.get('upscale_factor', 8)
        H = int(h * upscale_factor) // 16 * 16
        W = int(w * upscale_factor) // 16 * 16
    else:
        H, W = target_size
        H = H // 16 * 16
        W = W // 16 * 16

    first_stage_samples = first_stage_samples.permute(0, 3, 1, 2).to('cuda')

    ref_x = torch.nn.functional.interpolate(first_stage_samples,
                                            size=(H, W),
                                            mode='bilinear',
                                            align_corners=False,
                                            antialias=True)
    ref_x = ref_x[:num_frames][None]

    ref_x = ref_x.permute(0, 2, 1, 3, 4).contiguous()

    first_stage_model = model.first_stage_model
    print(f'start encoding first stage results to high resolution')
    with torch.no_grad():
        first_stage_dtype = next(model.first_stage_model.parameters()).dtype
        model.first_stage_model.cuda()
        ref_x = save_memory_encode_first_stage(
            ref_x.contiguous().to(first_stage_dtype).cuda(), model)

    ref_x = ref_x.permute(0, 2, 1, 3, 4).contiguous()
    ref_x = ref_x.to(model.dtype)
    print(f'finish encoding first stage results, and starting stage II')

    device = 'cuda'

    model.to(device)

    pos_prompt = ''
    input_negative_prompt = ""

    c, uc = prepare_input(text,
                          model,
                          num_frames,
                          negative_prompt=input_negative_prompt,
                          pos_prompt=pos_prompt)

    T = 1 + (num_frames - 1) // 4
    F = 8
    with torch.no_grad(), torch.amp.autocast(enabled=True,
                                             device_type='cuda',
                                             dtype=torch.bfloat16):
        samples_z = model.sample(
            ref_x,
            c,
            uc=uc,
            batch_size=1,
            shape=(T, 16, H // F, W // F),
            num_steps=model.share_cache.get('sample_step', 5),
            method='euler',
            cfg=model.share_cache.get('cfg', 7.5),
        )
        samples_z = samples_z.permute(0, 2, 1, 3, 4).contiguous()

        model.to('cpu')
        torch.cuda.empty_cache()
        first_stage_model = model.first_stage_model
        first_stage_model = first_stage_model.to(device)

        latent = 1.0 / model.scale_factor * samples_z
        print(f'start spatiotemporal slice decoding')
        samples = save_mem_decode(first_stage_model, latent)
        print(f'finish spatiotemporal slice decoding')
        model.to('cpu')
    return samples
    

In [ ]:





os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["RANK"] = "0"
os.environ["MASTER_ADDR"] = "0.0.0.0"
os.environ["MASTER_PORT"] = "12345"

py_parser = argparse.ArgumentParser(add_help=False)
args_list = [
    "--base", "flashvideo/configs/stage1.yaml",
    "--second", "flashvideo/configs/stage2.yaml",
    "--inf-ckpt", stage_1_path,
    "--inf-ckpt2", stage_2_path,
]
known, args_list = py_parser.parse_known_args(args=args_list)
second_args_list = copy.deepcopy(args_list)


args = get_args(args_list)
args = argparse.Namespace(**vars(args), **vars(known))
del args.deepspeed_config
args.model_config.first_stage_config.params.cp_size = 1
args.model_config.network_config.params.transformer_args.model_parallel_size = 1
args.model_config.network_config.params.transformer_args.checkpoint_activations = False
args.model_config.loss_fn_config.params.sigma_sampler_config.params.uniform_sampling = False

second_args_list[1] = args.second[0]
second_args = get_args(second_args_list)
second_args = argparse.Namespace(**vars(second_args), **vars(known))
del second_args.deepspeed_config
second_args.model_config.first_stage_config.params.cp_size = 1
second_args.model_config.network_config.params.transformer_args.model_parallel_size = 1
second_args.model_config.network_config.params.transformer_args.checkpoint_activations = False
second_args.model_config.loss_fn_config.params.sigma_sampler_config.params.uniform_sampling = False


In [ ]:
model_cls=SATVideoDiffusionEngine
second_model_cls=FlowEngine
local_rank = int(os.environ.get("LOCAL_RANK", 0))
torch.cuda.set_device(local_rank)

second_model = get_model(second_args, second_model_cls)

model = get_model(args, model_cls)
    
init_model(model, second_model, args, second_args )
    
model.eval()
second_model.eval()


In [ ]:
for n, m in model.named_modules():
    if hasattr(m, "merge_lora"):
        m.merge_lora()
        print(f"merge lora of {n}")

In [ ]:
num_frames = 49
second_num_frames = 49 

stage_1_hw = (270, 480)  
stage_2_hw = (1080, 1920) 

# make sure all can be divided by 16
stage_1_hw = (stage_1_hw[0] // 16 * 16, stage_1_hw[1] // 16 * 16)
stage_2_hw = (stage_2_hw[0] // 16 * 16, stage_2_hw[1] // 16 * 16)

sample_func = model.sample
T, H, W, C, F = num_frames, stage_1_hw[0], stage_1_hw[1], args.latent_channels, 8
S_T, S_H, S_W, S_C, S_F = second_num_frames, stage_2_hw[0], stage_2_hw[1], args.latent_channels, 8


    
seed_everything(0)

text = " Sunny day, The camera smoothly pushes in through an ornate garden archway, delicately adorned with climbing ivy. \
    Beyond the archway, a secret, tranquil garden is revealed, brimming with a vibrant array of blooming flowers \
        in a myriad of colors. A beautiful young woman with long wavy brown hair, she is smile to the camera , \
    wearing a red hat sits  holding a dog , the red hat has rich fabric texture  \
        wearing black pleated skirt and yellow sweater "


neg_text = ""

if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)
enu_index = "1"
model.share_cache["cfg"] = cfg_first

first_stage_samples = get_first_results(model, text, num_frames, H, W, neg_text)

print(f"save to {save_dir}/{enu_index}_num_frame_{num_frames}.mp4")
write_video(filename=f'./{save_dir}/{enu_index}_num_frame_{num_frames}.mp4', 
                fps=8, 
                video_array= first_stage_samples, 
                options = { 'crf': '14' })


In [ ]:


second_num_frames = 49
second_model.share_cache["ref_noise_step"] = deg_latent_strength
second_model.share_cache["sample_ref_noise_step"] = deg_latent_strength
second_model.share_cache.pop("ref_noise_step_range", None)
second_model.share_cache["target_size"] = stage_2_hw
second_model.share_cache["shift_t"] = shift_t
second_model.share_cache["sample_step"] = sample_step
second_model.share_cache["cfg"] = cfg_second
post_fix = f'''noise_{second_model.share_cache["ref_noise_step"]}_step_{second_model.share_cache["sample_step"]}_cfg_{second_model.share_cache["cfg"]}_shift_{second_model.share_cache["shift_t"]}_size_{stage_2_hw[0]}x{stage_2_hw[1]}'''
second_model.share_cache["time_size_embedding"] = True
second_stage_samples = get_second_results(second_model, 
                                            text, 
                                            first_stage_samples, 
                                            second_num_frames)

print(f"save to {save_dir}/{enu_index}_num_frame_{num_frames}_{post_fix}.mp4")
write_video(filename=f'./{save_dir}/{enu_index}_num_frame_{num_frames}_{post_fix}_second.mp4', 
                fps=8, 
                video_array= second_stage_samples.cpu(), 
                options = { 'crf': '14' })


# save joint video 
part_first_stage = first_stage_samples[:second_num_frames]

target_h, target_w = second_stage_samples.shape[1], second_stage_samples.shape[2]
part_first_stage = torch.nn.functional.interpolate(part_first_stage.permute(0, 3, 1, 2).contiguous(),
                                                    size=(target_h, target_w),
                                                    mode="bilinear",
                                                    align_corners=False, 
                                                    antialias=True)
part_first_stage = part_first_stage.permute(0, 2, 3, 1).contiguous()


joint_video = torch.cat([part_first_stage.cpu(), second_stage_samples.cpu()], dim=-2)
print(f'./{save_dir}/{enu_index}_num_frame_{num_frames}_{post_fix}_joint.mp4')
write_video(filename=f'./{save_dir}/{enu_index}_num_frame_{num_frames}_{post_fix}_joint.mp4',
                fps=8,
                video_array=joint_video.cpu(),
                options={'crf': '15'})   
